In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, BatchNormalization, Flatten, Activation, LayerNormalization, Reshape
from keras.optimizers import Adam

df = pd.read_csv('/kaggle/input/suduku/sudoku.csv')

X = np.array(df.quizzes.map(lambda x: list(map(int, x))).to_list())
Y = np.array(df.solutions.map(lambda x: list(map(int, x))).to_list())


X = X.reshape(-1, 9, 9, 1)
Y = Y.reshape(-1, 9, 9) - 1

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

model = Sequential()

model.add(Conv2D(128, 3, activation='relu', padding='same', input_shape = (9, 9, 1)))
model.add(BatchNormalization())
model.add(Conv2D(128, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(1024, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(9, 3, activation='relu', padding='same'))
model.add(Flatten())
model.add(Dense(512))
model.add(Dense(81*9))
model.add(LayerNormalization(axis=-1))
model.add(Reshape((9, 9, 9)))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

model.summary()

model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

model.evaluate(x_test, y_test)

model.save('/kaggle/working/mindlock-model.h5')







/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 9, 9, 128)         1280      
                                                                 
 batch_normalization (Batch  (None, 9, 9, 128)         512       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 128)         147584    
                                                                 
 batch_normalization_1 (Bat  (None, 9, 9, 128)         512       
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 256)         295168    
                                                                 
 batch_normalization_2 (Bat  (None, 9, 9, 256)         1

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
model = tf.keras.models.load_model('/kaggle/working/mindlock-model.h5')

sample_sudoku = np.array([0,0,4,3,0,0,2,0,9,0,0,5,0,0,9,0,0,1,0,7,0,0,6,0,0,4,3,0,0,6,0,0,2,0,8,7,1,9,0,0,0,7,4,0,0,0,5,0,0,8,3,0,0,0,6,0,0,0,0,0,1,0,5,0,0,3,5,0,8,6,9,0,0,4,2,9,1,0,3,0,0])

sample_sudoku = sample_sudoku.reshape(-1, 9, 9, 1)

solution_sudoku = model.predict([sample_sudoku])
solution_sudoku.argmax(-1)+1

1/1 [==============================] - 0s 398ms/step


array([[[7, 8, 6, 6, 6, 8, 8, 9, 7],
        [8, 8, 8, 8, 8, 8, 8, 8, 8],
        [8, 8, 8, 6, 6, 8, 8, 8, 8],
        [8, 8, 6, 6, 8, 8, 8, 8, 7],
        [8, 8, 8, 8, 8, 8, 8, 8, 8],
        [8, 8, 8, 6, 8, 8, 8, 8, 6],
        [6, 8, 8, 6, 8, 8, 8, 8, 8],
        [6, 8, 8, 6, 6, 8, 8, 8, 8],
        [7, 8, 8, 8, 6, 8, 8, 8, 7]]])